In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats import norm

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.cluster as cluster

In [2]:
data = pd.read_csv('../credit_card_kaggle/creditcard.csv')
data.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [3]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
data_cols = list(data.columns[ data.columns != 'Class' ])
label_cols = ['Class']

In [5]:
stdscaler=StandardScaler()
stdscaler.fit(data[data_cols])
data[data_cols] = stdscaler.transform(data[data_cols])

In [6]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.996583,-0.694242,-0.044075,1.672773,0.973366,-0.245117,0.347068,0.193679,0.082637,0.331128,...,-0.024923,0.382854,-0.176911,0.110507,0.246585,-0.392170,0.330892,-0.063781,0.244964,0
1,-1.996583,0.608496,0.161176,0.109797,0.316523,0.043483,-0.061820,-0.063700,0.071253,-0.232494,...,-0.307377,-0.880077,0.162201,-0.561131,0.320694,0.261069,-0.022256,0.044608,-0.342475,0
2,-1.996562,-0.693500,-0.811578,1.169468,0.268231,-0.364572,1.351454,0.639776,0.207373,-1.378675,...,0.337632,1.063358,1.456320,-1.138092,-0.628537,-0.288447,-0.137137,-0.181021,1.160686,0
3,-1.996562,-0.493325,-0.112169,1.182516,-0.609727,-0.007469,0.936150,0.192071,0.316018,-1.262503,...,-0.147443,0.007267,-0.304777,-1.941027,1.241904,-0.460217,0.155396,0.186189,0.140534,0
4,-1.996541,-0.591330,0.531541,1.021412,0.284655,-0.295015,0.071999,0.479302,-0.226510,0.744326,...,-0.012839,1.100011,-0.220123,0.233250,-0.395202,1.041611,0.543620,0.651816,-0.073403,0


In [7]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0)

In [8]:
def get_generator_block(input_dim, output_dim):
    '''
    Function for returning a block of the generator's neural network
    given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a generator neural network layer, with a linear transformation 
          followed by a batch normalization and then a relu activation
    '''
    return nn.Sequential(
        # https://pytorch.org/docs/stable/nn.html.
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )

In [9]:
# Verify the generator block function
def test_gen_block(in_features, out_features, num_test=1000):
    block = get_generator_block(in_features, out_features)

    # Check the three parts
    assert len(block) == 3
    assert type(block[0]) == nn.Linear
    assert type(block[1]) == nn.BatchNorm1d
    assert type(block[2]) == nn.ReLU
    
    # Check the output shape
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)
    assert tuple(test_output.shape) == (num_test, out_features)
    assert test_output.std() > 0.55
    assert test_output.std() < 0.65

test_gen_block(25, 12)
test_gen_block(15, 28)
print("Success!")

Success!


In [10]:
#  Generator
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
          (MNIST images are 28 x 28 = 784 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, z_dim=10, im_dim=30, hidden_dim=20):
        super(Generator, self).__init__()
        # Build the neural network
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            # There is a dropdown with hints if you need them!
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor, 
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        return self.gen(noise)
    
    # Needed for grading
    def get_gen(self):
        '''
        Returns:
            the sequential model
        '''
        return self.gen

In [11]:
# Verify the generator class
def test_generator(z_dim, im_dim, hidden_dim, num_test=10000):
    gen = Generator(z_dim, im_dim, hidden_dim).get_gen()
    
    # Check there are six modules in the sequential part
    assert len(gen) == 6
    assert str(gen.__getitem__(4)).replace(' ', '') == f'Linear(in_features={hidden_dim * 8},out_features={im_dim},bias=True)'
    assert str(gen.__getitem__(5)).replace(' ', '') == 'Sigmoid()'
    test_input = torch.randn(num_test, z_dim)
    test_output = gen(test_input)

    # Check that the output shape is correct
    assert tuple(test_output.shape) == (num_test, im_dim)
    assert test_output.max() < 1, "Make sure to use a sigmoid"
    assert test_output.min() > 0, "Make sure to use a sigmoid"
    assert test_output.std() > 0.05, "Don't use batchnorm here"
    assert test_output.std() < 0.15, "Don't use batchnorm here"

test_generator(5, 10, 20)
test_generator(20, 8, 24)
print("Success!")

Success!


In [12]:
def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim),
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    '''
    # NOTE: To use this on GPU with device='cuda', make sure to pass the device 
    # argument to the function you use to generate the noise.
    return torch.randn(n_samples,z_dim,device=device)


In [13]:
# Verify the noise vector function
def test_get_noise(n_samples, z_dim, device='cpu'):
    noise = get_noise(n_samples, z_dim, device)
    # Make sure a normal distribution was used
    assert tuple(noise.shape) == (n_samples, z_dim)
    assert torch.abs(noise.std() - torch.tensor(1.0)) < 0.01
    assert str(noise.device).startswith(device)

test_get_noise(1000, 100, 'cpu')
if torch.cuda.is_available():
    test_get_noise(1000, 32, 'cuda')
print("Success!")

Success!


In [14]:
# get_discriminator_block
def get_discriminator_block(input_dim, output_dim):
    '''
    Discriminator Block
    Function for returning a neural network of the discriminator given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a discriminator neural network layer, with a linear transformation 
          followed by an nn.LeakyReLU activation with negative slope of 0.2 
          (https://pytorch.org/docs/master/generated/torch.nn.LeakyReLU.html)
    '''
    return nn.Sequential(
         nn.Linear(input_dim, output_dim), #Layer 1
         nn.LeakyReLU(0.2, inplace=True)
    )

In [15]:
#  Verify the discriminator block function
def test_disc_block(in_features, out_features, num_test=10000):
    block = get_discriminator_block(in_features, out_features)

    # Check there are two parts
    assert len(block) == 2
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)

    # Check that the shape is right
    assert tuple(test_output.shape) == (num_test, out_features)
    
    # Check that the LeakyReLU slope is about 0.2
    assert -test_output.min() / test_output.max() > 0.1
    assert -test_output.min() / test_output.max() < 0.3
    assert test_output.std() > 0.3
    assert test_output.std() < 0.5
    
    assert str(block.__getitem__(0)).replace(' ', '') == f'Linear(in_features={in_features},out_features={out_features},bias=True)'        
    assert str(block.__getitem__(1)).replace(' ', '').replace(',inplace=True', '') == 'LeakyReLU(negative_slope=0.2)'


test_disc_block(25, 12)
test_disc_block(15, 28)
print("Success!")

Success!


In [16]:
# GRADED FUNCTION: Discriminator
class Discriminator(nn.Module):
    '''
    Discriminator Class
    Values:
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
            (MNIST images are 28x28 = 784 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, im_dim=30, hidden_dim=10):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            # Hint: You want to transform the final output into a single value,
            #       so add one more linear map.
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image):
        '''
        Function for completing a forward pass of the discriminator: Given an image tensor, 
        returns a 1-dimension tensor representing fake/real.
        Parameters:
            image: a flattened image tensor with dimension (im_dim)
        '''
        return self.disc(image.float())
    
    # Needed for grading
    def get_disc(self):
        '''
        Returns:
            the sequential model
        '''
        return self.disc

In [17]:
# Verify the discriminator class
def test_discriminator(z_dim, hidden_dim, num_test=100):
    
    disc = Discriminator(z_dim, hidden_dim).get_disc()

    # Check there are three parts
    assert len(disc) == 4
    assert type(disc.__getitem__(3)) == nn.Linear

    # Check the linear layer is correct
    test_input = torch.randn(num_test, z_dim)
    test_output = disc(test_input)
    assert tuple(test_output.shape) == (num_test, 1)

test_discriminator(5, 10)
test_discriminator(20, 8)
print("Success!")

Success!


In [18]:
# Set your parameters
criterion = nn.BCEWithLogitsLoss()
n_epochs = 10
z_dim = 10
hidden_dim=20
display_step = 500
batch_size = 16
lr = 0.00001
device = 'cpu'

In [19]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)

In [20]:
# get_disc_loss
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    '''
    Return the loss of the discriminator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        real: a batch of real images
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        disc_loss: a torch scalar loss value for the current batch
    '''
    #     These are the steps you will need to complete:
    #       1) Create noise vectors and generate a batch (num_images) of fake images. 
    #            Make sure to pass the device argument to the noise.
    #       2) Get the discriminator's prediction of the fake image 
    #            and calculate the loss. Don't forget to detach the generator!
    #            (Remember the loss function you set earlier -- criterion. You need a 
    #            'ground truth' tensor in order to calculate the loss. 
    #            For example, a ground truth tensor for a fake image is all zeros.)
    #       3) Get the discriminator's prediction of the real image and calculate the loss.
    #       4) Calculate the discriminator's loss by averaging the real and fake loss
    #            and set it to disc_loss.

    fake_noise = get_noise(num_images, z_dim, device=device)
    fake = gen(fake_noise)
    disc_fake_pred = disc(fake.detach())
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
    disc_loss = (disc_fake_loss + disc_real_loss) / 2
    return disc_loss

In [21]:
features= data[data_cols]
type(features)

pandas.core.frame.DataFrame

In [22]:
targets = data[['Class']]
type(targets)

pandas.core.frame.DataFrame

In [23]:
from torch.utils.data import DataLoader, TensorDataset

train = TensorDataset(torch.tensor(features.values,dtype=torch.float64), torch.tensor(targets.values,dtype=torch.float64))
train_loader = DataLoader(train, batch_size=16, shuffle=True)



In [24]:
dataloader=DataLoader(train, batch_size=16, shuffle=True)

In [25]:
def test_disc_reasonable(num_images=10):
    z_dim = 64
    gen = torch.zeros_like
    disc = nn.Identity()
    criterion = torch.mul # Multiply
    real = torch.ones(num_images, 1)
    disc_loss = get_disc_loss(gen, disc, criterion, real, num_images, z_dim, 'cpu')
    assert tuple(disc_loss.shape) == (num_images, z_dim)
    assert torch.all(torch.abs(disc_loss - 0.5) < 1e-5)

    gen = torch.ones_like
    disc = nn.Identity()
    criterion = torch.mul # Multiply
    real = torch.zeros(num_images, 1)
    assert torch.all(torch.abs(get_disc_loss(gen, disc, criterion, real, num_images, z_dim, 'cpu')) < 1e-5)

def test_disc_loss(max_tests = 10):
    z_dim = 64
    gen = Generator(z_dim).to(device)
    gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
    disc = Discriminator().to(device) 
    disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)
    num_steps = 0
    for real in dataloader:
        real=real[0]
        cur_batch_size = len(real)
        real = real.view(cur_batch_size, -1).to(device)
#         real=torch.tensor(real).view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradient before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)
        assert (disc_loss - 0.68).abs() < 0.05

        # Update gradients
        disc_loss.backward(retain_graph=True)

        # Check that they detached correctly
        assert gen.gen[0][0].weight.grad is None

        # Update optimizer
        old_weight = disc.disc[0][0].weight.data.clone()
        disc_opt.step()
        new_weight = disc.disc[0][0].weight.data
        
        # Check that some discriminator weights changed
        assert not torch.all(torch.eq(old_weight, new_weight))
        num_steps += 1
        if num_steps >= max_tests:
            break

test_disc_reasonable()
test_disc_loss()
print("Success!")

Success!


In [26]:
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    '''
    Return the loss of the generator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        gen_loss: a torch scalar loss value for the current batch
    '''
    #     These are the steps you will need to complete:
    #       1) Create noise vectors and generate a batch of fake images. 
    #           Remember to pass the device argument to the get_noise function.
    #       2) Get the discriminator's prediction of the fake image.
    #       3) Calculate the generator's loss. Remember the generator wants
    #          the discriminator to think that its fake images are real
    #     *Important*: You should NOT write your own loss function here - use criterion(pred, true)!

    fake_noise = get_noise(num_images, z_dim, device=device)
    fake = gen(fake_noise)
    disc_fake_pred = disc(fake)
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    return gen_loss

In [27]:
def test_gen_reasonable(num_images=10):
    z_dim = 64
    gen = torch.zeros_like
    disc = nn.Identity()
    criterion = torch.mul # Multiply
    gen_loss_tensor = get_gen_loss(gen, disc, criterion, num_images, z_dim, 'cpu')
    assert torch.all(torch.abs(gen_loss_tensor) < 1e-5)
    #Verify shape. Related to gen_noise parametrization
    assert tuple(gen_loss_tensor.shape) == (num_images, z_dim)

    gen = torch.ones_like
    disc = nn.Identity()
    criterion = torch.mul # Multiply
    real = torch.zeros(num_images, 1)
    gen_loss_tensor = get_gen_loss(gen, disc, criterion, num_images, z_dim, 'cpu')
    assert torch.all(torch.abs(gen_loss_tensor - 1) < 1e-5)
    #Verify shape. Related to gen_noise parametrization
    assert tuple(gen_loss_tensor.shape) == (num_images, z_dim)
    

def test_gen_loss(num_images):
    z_dim = 64
    gen = Generator(z_dim).to(device)
    gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
    disc = Discriminator().to(device) 
    disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)
    
    gen_loss = get_gen_loss(gen, disc, criterion, num_images, z_dim, device)
    
    # Check that the loss is reasonable
    assert (gen_loss - 0.7).abs() < 0.1
    gen_loss.backward()
    old_weight = gen.gen[0][0].weight.clone()
    gen_opt.step()
    new_weight = gen.gen[0][0].weight
    assert not torch.all(torch.eq(old_weight, new_weight))


test_gen_reasonable(10)
test_gen_loss(18)
print("Success!")

Success!


In [28]:
from numpy import dot
from numpy.linalg import norm

In [29]:
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
test_generator = True # Whether the generator should be tested
gen_loss = False
error = False
for epoch in range(n_epochs):
  
    # Dataloader returns the batches
    for real, _ in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        real = real.view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradients before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)

        # Update gradients
        disc_loss.backward(retain_graph=True)

        # Update optimizer
        disc_opt.step()

        # For testing purposes, to keep track of the generator weights
        if test_generator:
            old_generator_weights = gen.gen[0][0].weight.detach().clone()

        ### Update generator ###
        #     Hint: This code will look a lot like the discriminator updates!
        #     These are the steps you will need to complete:
        #       1) Zero out the gradients.
        #       2) Calculate the generator loss, assigning it to gen_loss.
        #       3) Backprop through the generator: update the gradients and optimizer.
        #### START CODE HERE ####
        gen_opt.zero_grad()
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
        gen_loss.backward()
        gen_opt.step()
        #### END CODE HERE ####

        # For testing purposes, to check that your code changes the generator weights
        if test_generator:
            try:
                assert lr > 0.0000002 or (gen.gen[0][0].weight.grad.abs().max() < 0.0005 and epoch == 0)
                assert torch.any(gen.gen[0][0].weight.detach().clone() != old_generator_weights)
            except:
                error = True
                print("Runtime tests have failed")

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1

  0%|          | 0/17801 [00:00<?, ?it/s]

Step 500: Generator loss: 0.6993939344882961, discriminator loss: 0.6979003940820702
Step 1000: Generator loss: 0.7080386548042296, discriminator loss: 0.6858605178594595
Step 1500: Generator loss: 0.7182726676464074, discriminator loss: 0.6748698900938033
Step 2000: Generator loss: 0.7272632989883419, discriminator loss: 0.664324694752693
Step 2500: Generator loss: 0.735707288622856, discriminator loss: 0.6534168376922608
Step 3000: Generator loss: 0.7447998908758157, discriminator loss: 0.6413002980947492
Step 3500: Generator loss: 0.7514740092754356, discriminator loss: 0.631210120916366
Step 4000: Generator loss: 0.7552924493551253, discriminator loss: 0.6225622603893278
Step 4500: Generator loss: 0.7617049475908284, discriminator loss: 0.6124617586135862
Step 5000: Generator loss: 0.7714958199262617, discriminator loss: 0.5994006869792943
Step 5500: Generator loss: 0.7858519797325138, discriminator loss: 0.5845355319976809
Step 6000: Generator loss: 0.804184844970704, discriminato

  0%|          | 0/17801 [00:00<?, ?it/s]

Step 18000: Generator loss: 2.1308159518241876, discriminator loss: 0.09077370567619804
Step 18500: Generator loss: 2.191973841667177, discriminator loss: 0.08498991207778445
Step 19000: Generator loss: 2.2711414213180574, discriminator loss: 0.08049246792495243
Step 19500: Generator loss: 2.407256175518036, discriminator loss: 0.07016036789119247
Step 20000: Generator loss: 2.5344805245399473, discriminator loss: 0.06095814440399411
Step 20500: Generator loss: 2.65186920404434, discriminator loss: 0.0535435343980789
Step 21000: Generator loss: 2.7722719902992243, discriminator loss: 0.04778721243143084
Step 21500: Generator loss: 2.9272347812652595, discriminator loss: 0.040323337152600294
Step 22000: Generator loss: 3.0780123586654637, discriminator loss: 0.03473978361859921
Step 22500: Generator loss: 3.2266921567916858, discriminator loss: 0.029447516169399026
Step 23000: Generator loss: 3.3758419923782337, discriminator loss: 0.025352543197572224
Step 23500: Generator loss: 3.5384

  0%|          | 0/17801 [00:00<?, ?it/s]

...

Step 125000: Generator loss: 8.906205696105959, discriminator loss: 0.00012525196610658885
Step 125500: Generator loss: 8.995532814025882, discriminator loss: 0.00012478916451073025
Step 126000: Generator loss: 9.080310636520377, discriminator loss: 0.0001480824395548552
Step 126500: Generator loss: 9.063779481887822, discriminator loss: 0.00011169582125148742
Step 127000: Generator loss: 9.151626085281375, discriminator loss: 9.535020703333422e-05
Step 127500: Generator loss: 9.16340373611451, discriminator loss: 0.00025287924092117463
Step 128000: Generator loss: 9.314557550430305, discriminator loss: 7.646446240687499e-05
Step 128500: Generator loss: 9.374814405441285, discriminator loss: 0.0002326942194413277
Step 129000: Generator loss: 9.342246852874757, discriminator loss: 6.651421429705808e-05
Step 129500: Generator loss: 9.415981567382815, discriminator loss: 9.921057476458373e-05
Step 130000: Generator loss: 9.471274404525767, discriminator loss: 9.305274429061679e-05
Step 13

  0%|          | 0/17801 [00:00<?, ?it/s]

Step 142500: Generator loss: 9.64136783409118, discriminator loss: 0.00012003475741585142
Step 143000: Generator loss: 9.676340110778808, discriminator loss: 7.311202804703494e-05
Step 143500: Generator loss: 9.76557581710815, discriminator loss: 7.892244411414139e-05
Step 144000: Generator loss: 9.70099215126038, discriminator loss: 0.00012281203069142063
Step 144500: Generator loss: 9.411455114364612, discriminator loss: 0.0003423388913761301
Step 145000: Generator loss: 9.176090270996095, discriminator loss: 0.00011326139526499903
Step 145500: Generator loss: 9.300341924667357, discriminator loss: 0.00016832525591598817
Step 146000: Generator loss: 9.347526535034177, discriminator loss: 0.00010246773176186252
Step 146500: Generator loss: 9.40922862625122, discriminator loss: 0.00011383433989976771
Step 147000: Generator loss: 9.386606513977055, discriminator loss: 0.00026158602979558057
Step 147500: Generator loss: 9.32380242156983, discriminator loss: 0.00019579224674816948
Step 14

  0%|          | 0/17801 [00:00<?, ?it/s]

Step 160500: Generator loss: 9.88571085357666, discriminator loss: 4.917359061437307e-05
Step 161000: Generator loss: 10.023953302383411, discriminator loss: 3.8608257054875135e-05
Step 161500: Generator loss: 10.064488235473636, discriminator loss: 6.509230570372899e-05
Step 162000: Generator loss: 10.152782327651975, discriminator loss: 3.684449191132443e-05
Step 162500: Generator loss: 10.19467779731751, discriminator loss: 4.796321269168402e-05
Step 163000: Generator loss: 10.177417753219602, discriminator loss: 3.414452121796787e-05
Step 163500: Generator loss: 10.321515624999991, discriminator loss: 2.808853533497311e-05
Step 164000: Generator loss: 10.371677616119397, discriminator loss: 3.194629259087376e-05
Step 164500: Generator loss: 10.27721604347229, discriminator loss: 5.1819057818647696e-05
Step 165000: Generator loss: 10.27135887527466, discriminator loss: 2.9952322765893755e-05
Step 165500: Generator loss: 10.356668500900259, discriminator loss: 3.931716732040515e-05
S

In [30]:
torch.save(gen, 'genmodel_1.pt')

In [31]:
torch.save(disc, 'discmodel_1.pt')

In [32]:
fake_noise = get_noise(cur_batch_size, z_dim, device=device)

In [33]:
fake = gen(fake_noise)

In [34]:
new_data=fake.detach().numpy()

In [35]:
new_df=pd.DataFrame(new_data)

In [38]:
stdscaler.inverse_transform(new_data)

array([[ 9.48138594e+04,  1.16837501e-15,  7.37445098e-13,
        -1.37951301e-15,  6.32896920e-14,  1.48169566e-09,
         7.34258834e-12,  3.35667154e-11,  1.21348508e-16,
        -2.40633061e-15,  2.23905282e-15,  1.02071059e+00,
        -1.24453492e-15,  2.34485012e-13,  1.71233936e-15,
         1.77745199e-12,  1.43771593e-15, -3.77217070e-16,
         1.75909440e-06,  6.56868223e-14,  1.69155818e-11,
         6.54433994e-03,  7.25700259e-01,  2.57873516e-16,
         6.05640829e-01,  2.00923235e-13,  1.68343722e-15,
         3.58247902e-08,  4.14030234e-16,  8.83496170e+01],
       [ 9.48138594e+04,  2.12514925e-14,  3.13835243e-12,
         5.73104769e-13,  3.04223322e-05,  1.38000154e+00,
         1.33226013e+00,  1.23621821e+00,  2.18783922e-10,
         7.35574169e-15,  1.30210225e-14,  3.96847863e-05,
         4.43718142e-14,  4.35258485e-14,  1.82315500e-11,
         8.74650297e-10,  4.78096092e-14,  1.45172994e-14,
         6.25565044e-10,  4.14026621e-14,  3.76515964e-

In [41]:
def create_new_data(n_times=5):
    data=[]
    for _ in range(n_times):
        fake_noise = get_noise(cur_batch_size, z_dim, device=device)
        fake = gen(fake_noise)
        new_data=fake.detach().numpy()
        data.append(stdscaler.inverse_transform(new_data))
    return pd.DataFrame(np.vstack(data))

create_new_data(n_times=1)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,94813.859375,1.145012e-14,1.798995e-12,2.698096e-13,2.214089e-05,1.380045e+00,1.332260e+00,1.236281e+00,1.234889e-10,2.382825e-15,...,2.367150e-04,8.518502e-08,0.000121,1.587450e-13,6.051279e-01,5.212645e-01,1.211188e-14,4.032198e-01,8.124181e-10,88.349617
1,94813.859375,1.168375e-15,6.321139e-12,-1.378310e-15,2.975302e-12,6.261858e-08,4.602014e-11,5.589284e-09,1.213853e-16,-2.406331e-15,...,1.016725e-09,1.399751e-02,0.725700,2.585282e-16,6.056389e-01,2.840503e-12,1.683437e-15,6.078702e-07,5.557251e-15,88.349617
2,94813.859375,1.168379e-15,1.163605e-11,-1.320732e-15,1.084141e-10,4.512992e-08,2.465428e-09,2.543273e-09,1.269183e-16,-2.406321e-15,...,3.001892e-09,2.714750e-02,0.725700,2.856627e-16,6.056342e-01,3.962971e-11,1.683453e-15,1.216383e-06,1.727578e-13,88.349617
3,94813.859375,1.168384e-15,1.279755e-11,-1.344555e-15,1.888343e-12,2.474307e-09,2.047639e-09,4.068998e-11,1.227499e-16,-2.406324e-15,...,4.627434e-11,1.290347e-02,0.725700,2.642858e-16,6.056412e-01,5.227355e-11,1.683458e-15,7.133251e-08,6.905088e-14,88.349617
4,94813.859375,1.168375e-15,1.235099e-12,-1.379411e-15,1.852833e-12,1.627774e-08,2.125558e-11,8.787300e-10,1.213534e-16,-2.406331e-15,...,3.910836e-10,1.577358e-02,0.725700,2.579465e-16,6.056389e-01,6.140811e-13,1.683437e-15,1.925502e-07,1.577614e-15,88.349617
5,94813.859375,5.268463e-15,8.423571e-13,1.303283e-13,1.541098e-05,1.380091e+00,1.332263e+00,1.236346e+00,6.819788e-11,-6.262313e-16,...,1.941581e-04,5.416538e-08,0.000098,6.227654e-14,6.052554e-01,5.212693e-01,5.393157e-15,4.032387e-01,3.823848e-10,88.349617
6,94813.859375,1.168375e-15,3.416913e-16,-1.379537e-15,2.074095e-15,1.430512e-09,2.162515e-15,3.202851e-13,1.213481e-16,-2.406331e-15,...,7.104779e-16,9.589246e-15,0.725700,2.578648e-16,2.702260e-07,5.343878e-16,1.683437e-15,2.194947e-09,-1.227390e-16,88.349617
